In [9]:
import pandas as pd

root = "C:\\Users\\user\\SadafPythonCode\\MLHackathon\\ML-for-Good-Hackathon\\Data\\"
df_p1 = pd.read_csv(root + 'ProlificAcademic\\April 2020\\Data\\CRISIS_Parent_April_2020.csv')
df_p2 = pd.read_csv(root + 'ProlificAcademic\\May 2020\\Data\\CRISIS_Parent_May_2020.csv')
df_p3 = pd.read_csv(root + 'ProlificAcademic\\November 2020\\Data\\CRISIS_Parent_November_2020.csv')
df_p4 = pd.read_csv(root + 'ProlificAcademic\\April 2021\\Data\\CRISIS_Parent_April_2021.csv')

In [10]:
df_pos_1 = df_p1[['specifypositive']].dropna()
df_pos_2 = df_p2[['specifypositive']].dropna()
df_pos_3 = df_p3[['specifypositive']].dropna()
df_pos_4 = df_p4[['specifypositive']].dropna()

df_combined = pd.concat([df_pos_1, df_pos_2, df_pos_3, df_pos_4])


In [11]:
# Load the regular expression library
import re

# Remove punctuation
df_combined['specifypositive_processed'] = \
df_combined['specifypositive'].map(lambda x: re.sub('[,\.!?]', '', x))

# Convert the titles to lowercase
df_combined['specifypositive_processed'] = \
df_combined['specifypositive_processed'].map(lambda x: x.lower())

# Print out the first rows of papers
df_combined['specifypositive_processed'].head()

3                                        dad home more
4    he enjoys having his father working at home al...
6                            full time at home with me
7    more time with her parents more time outside m...
9    i spend more time with them because i work in ...
Name: specifypositive_processed, dtype: object

In [25]:
import gensim
from gensim.utils import simple_preprocess

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data = df_combined.specifypositive_processed.values.tolist()
data_words = list(sent_to_words(data))

print(data_words[:1][0][:30])

['dad', 'home', 'more']


In [26]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

remove stop words, make bigrams and lemmatize

In [27]:
# NLTK Stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [31]:
import spacy

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

['dad', 'home']


In [32]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1)]


LDA model tranining

In [33]:
from pprint import pprint

# number of topics
num_topics = 10

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.372*"time" + 0.269*"family" + 0.075*"parent" + 0.051*"home" + '
  '0.046*"spend" + 0.027*"quality" + 0.012*"increase" + 0.009*"sible" + '
  '0.005*"develop" + 0.005*"slow"'),
 (1,
  '0.099*"close" + 0.059*"family" + 0.053*"relationship" + 0.033*"lockdown" + '
  '0.032*"get" + 0.031*"love" + 0.027*"well" + 0.024*"friend" + 0.024*"great" '
  '+ 0.020*"think"'),
 (2,
  '0.035*"play" + 0.032*"learning" + 0.029*"school" + 0.028*"online" + '
  '0.024*"focus" + 0.023*"money" + 0.021*"activity" + 0.020*"increase" + '
  '0.020*"child" + 0.019*"outside"'),
 (3,
  '0.065*"school" + 0.053*"go" + 0.029*"also" + 0.029*"day" + 0.025*"much" + '
  '0.023*"feel" + 0.021*"walk" + 0.019*"son" + 0.017*"home" + 0.015*"grow"'),
 (4,
  '0.068*"work" + 0.067*"exercise" + 0.032*"well" + 0.030*"school" + '
  '0.030*"become" + 0.027*"due" + 0.024*"improve" + 0.023*"get" + 0.021*"game" '
  '+ 0.021*"home"'),
 (5,
  '0.180*"time" + 0.161*"spending" + 0.132*"family" + 0.045*"together" + '
  '0.026*"year" +

In [34]:
from gensim.models import CoherenceModel

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.39600267175416426


In [35]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [37]:
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('./lda_tuning_results.csv', index=False)
    pbar.close()


100%|████████████████████████████████████████████████████████████████████████████████| 540/540 [48:52<00:00,  5.43s/it]

 12%|█████████▌                                                                     | 65/540 [07:02<1:01:51,  7.81s/it]


 24%|███████████████████▍                                                            | 131/540 [15:51<54:16,  7.96s/it]


 36%|█████████████████████████████▏                                                  | 197/540 [25:24<42:43,  7.47s/it]


 49%|██████████████████████████████████████▉                                         | 263/540 [32:57<30:02,  6.51s/it]


 61%|████████████████████████████████████████████████▋                               | 329/540 [40:25<25:21,  7.21s/it]


 73%|██████████████████████████████████████████████████████████▌                     | 395/540 [48:55<17:28,  7.23s/it]


 85%|████████████████████████████████████████████████████████████████████▎           | 461/540 [57:27<11:47,  8.95s/it]


 98%|████████████████████████████████████████████████████████████████████████████  | 527/540 [1:06:45<01:49,  8.44s/it]


100%|██████████████████████████████████████████████████████████████████████████████| 540/540 [1:08:40<00:00,  7.63s/it]


In [38]:

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.61,
                                           eta=0.01)

In [39]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.368*"time" + 0.192*"parent" + 0.088*"quality" + 0.042*"increase" + '
  '0.037*"family" + 0.031*"take" + 0.022*"love" + 0.021*"become" + '
  '0.018*"interest" + 0.017*"brother"'),
 (1,
  '0.439*"family" + 0.104*"close" + 0.096*"time" + 0.055*"relationship" + '
  '0.041*"bond" + 0.019*"lockdown" + 0.019*"immediate" + 0.019*"extra" + '
  '0.017*"self" + 0.012*"homeschooling"'),
 (2,
  '0.234*"school" + 0.089*"less" + 0.051*"go" + 0.044*"much" + 0.042*"enjoy" + '
  '0.035*"learning" + 0.034*"happy" + 0.027*"stress" + 0.025*"home" + '
  '0.022*"make"'),
 (3,
  '0.349*"spend" + 0.194*"time" + 0.073*"able" + 0.033*"home" + '
  '0.025*"outdoors" + 0.019*"together" + 0.017*"talk" + 0.016*"family" + '
  '0.015*"like" + 0.015*"start"'),
 (4,
  '0.153*"work" + 0.086*"exercise" + 0.058*"play" + 0.055*"time" + '
  '0.046*"walk" + 0.045*"due" + 0.027*"sleep" + 0.026*"free" + 0.025*"improve" '
  '+ 0.024*"daily"'),
 (5,
  '0.355*"time" + 0.286*"family" + 0.132*"together" + 0.110*"spending" +

visualize results

In [40]:
import pyLDAvis.gensim_models as gensimvis
import pickle 
import pyLDAvis
import os

# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_data_filepath = os.path.join('./ldavis_prepared_'+str(num_topics))

# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

pyLDAvis.save_html(LDAvis_prepared, './ldavis_prepared_'+ str(num_topics) +'.html')

LDAvis_prepared

C:\Users\user\SadafPythonCode\MLHackathon\mybert\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.269961 -0.045908       1        1  11.387554
2     -0.288934  0.113475       2        1  11.052956
8     -0.231252 -0.302751       3        1  10.292266
3      0.089151  0.013278       4        1  10.022987
0      0.178992 -0.034166       5        1   9.740766
9     -0.195643 -0.117640       6        1   9.656906
7     -0.034272  0.191893       7        1   9.640614
6      0.086856  0.079319       8        1   9.600067
4     -0.093356  0.210432       9        1   9.567485
1      0.218496 -0.107932      10        1   9.038398, topic_info=         Term         Freq        Total Category  logprob  loglift
6      family  1146.000000  1146.000000  Default  30.0000  30.0000
23      spend   512.000000   512.000000  Default  29.0000  29.0000
12       time  1981.000000  1981.000000  Default  28.0000  28.0000
1        home   690.000000   690.000000  Default  27.0000  27.0000
22     school   388.000000   388.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
476      self    20.920627    21.968289  Topic10  -4.0959   2.3548
6      family   552.476585  1146.095442  Topic10  -0.8223   1.6740
304   benefit     6.989826     8.263197  Topic10  -5.1922   2.2363
524  lockdown    24.356737    36.196873  Topic10  -3.9439   2.0075
12       time   120.729694  1981.567644  Topic10  -2.3431  -0.3944

[444 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
413       7  1.024738  ability
113       3  0.038014     able
113       4  0.775489     able
113       5  0.007603     able
113       6  0.136851     able
...     ...       ...      ...
258       3  0.930061     yard
296       3  0.307071     year
296       5  0.716500     year
390       6  0.990117    young
605       3  0.949612     zoom

[509 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 3, 9, 4, 1, 10, 8, 7, 5, 2])